In [9]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow import keras
import datetime
from pathlib import Path
import os
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping


In [10]:
train_ds = h5py.File("train_dataset.h5", 'r')
# for i in train_ds['x_dataset']:
#     print(i.reshape((-1, 257)).shape)

In [11]:
# class AudioSequenceGenerator():


In [16]:
from tensorflow.keras.layers import (BatchNormalization, Conv2D, Dense,
                                     Dropout, Flatten, Input, MaxPool2D, LSTM, TimeDistributed)
from tensorflow.keras.models import Model

def build_model(input_tensor):
    # create model
    input = Input(shape=input_tensor.shape[1:])
    hid1 = LSTM(n_units,return_sequences=True, activation='relu')(input)
    dp1  = Dropout(0.2)(hid1)
    hid2 = LSTM(n_units,return_sequences=True, activation='relu')(dp1)
    dp2  = Dropout(0.2)(hid2)
    hid3 = LSTM(n_units,return_sequences=True, activation='relu')(dp2)
    y1_hat = TimeDistributed(Dense(input_tensor.shape[2], activation='softmax', input_shape=input_tensor.shape[1:]), name='y1_hat')(hid3)

    model = Model(inputs=input,outputs=y1_hat)
    return model, input

In [17]:
def spectrumSequence(nfft):
    nFiles = 200
    sequence = []
    lengths = []
    for idx in range(nFiles):
        x_len = np.random.randint(low=100, high=1000)
        Mag = np.random.random((x_len, nfft))
        sequence.append(Mag)
        lengths.append(len(Mag))
    return sequence,lengths

from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_seq(allData,maxlen):
    paddedData = pad_sequences(allData,maxlen=maxlen,dtype='float32',value=0.0)
    return paddedData

In [14]:
nfft = 257
x_data,l1  = spectrumSequence(nfft)
y1_data,l2 = spectrumSequence(nfft)

assert len(x_data) == len(y1_data)

l1 = max(l1)
l2 = max(l2)
maxL = max(l1,l2)

train_x = pad_seq(x_data,maxL)
y1      = pad_seq(y1_data,maxL)

# # Model Parameters
batch_size = 10
learning_rate = 1e-5
decay_ = 1e-3
epochs = 200
n_units = int(2*nfft/1)


from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(train_x, y1, test_size=0.2, random_state=42)

In [18]:
model, input_layer = build_model(X_train)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 999, 257)]        0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 999, 514)          1587232   
_________________________________________________________________
dropout (Dropout)            (None, 999, 514)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 999, 514)          2115624   
_________________________________________________________________
dropout_1 (Dropout)          (None, 999, 514)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 999, 514)          2115624   
_________________________________________________________________
y1_hat (TimeDistributed)     (None, 999, 257)          1323

In [26]:
import keras.backend as K
import tensorflow.keras.backend as K

# Define custom loss
def custom_loss(layer):

    # Create a loss function based on MSE
    def loss(y_true,y_pred):
        # y_out = tf.math.multiply(layer, y_pred)
        err = K.mean(K.square(y_pred - y_true), axis=-1)
        return err

    # Return a function
    return loss

In [33]:
# Compile the model using Adam's default learning rate
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate, decay=decay_)

model.compile(loss=custom_loss(input_layer),optimizer=opt, metrics=['accuracy'])

model_save_filename = "model.h5"

ct = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
Path("models/" + str(ct)).mkdir(parents=True, exist_ok=True)

# Add callbacks:
# 'EarlyStopping' to stop training when the model is not enhancing anymore
# 'ModelCheckPoint' to always keep the model that has the best val_accuracy
earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

curdir = os.getcwd()+"/logs/"+str(ct)
tensorboard = TensorBoard(log_dir=curdir)

In [34]:
shape = X_train.shape[1:]
n_outs = X_train.shape[2] # Note: Not train_x.shape[1:], which returns shape for input_shape, instead of int.

history = model.fit(
    x=X_train,
    y=Y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, Y_test),
    callbacks=[tensorboard,mdlcheckpoint_cb,earlystopping_cb]
)

Epoch 1/200
 1/16 [>.............................] - ETA: 29:51 - loss: 0.2115 - accuracy: 0.0020

KeyboardInterrupt: 